In [47]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from io import StringIO
import datetime
from sklearn.preprocessing import normalize

now = datetime.datetime.now().strftime(format="%Y-%m-%d_%H-%M")

# Load training data
df_train = pd.read_csv('../data/train-sep.csv')

# Separation des donnees et pretraitement
X_train_raw = df_train['titre']
y_train_raw = df_train['type']

# Preprocess the training text data
preprocessed_text_train = [simple_preprocess(sentence) for sentence in X_train_raw]

# Train Word2Vec model
model = Word2Vec(sentences=preprocessed_text_train, vector_size=100, window=5, min_count=1, workers=10)

# Transform text data to Word2Vec embeddings
#X_train = np.array([np.mean([model.wv[word] for word in words if word in model.wv] or [np.zeros(100)], axis=0) for words in preprocessed_text_train])

# Encode the labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_raw)

# Load test data
df_test = pd.read_csv('../data/test.csv')
X_test = df_test['titre']

# Preprocess the test text data
preprocessed_text_test = [simple_preprocess(sentence) for sentence in X_test]

# Transform test text data to Word2Vec embeddings
X_test_emb = np.array([np.mean([model.wv[word] for word in words if word in model.wv] or [np.zeros(100)], axis=0) for words in preprocessed_text_test])

# Train SVM classifier
classifier = SVC()
classifier.fit(X_train, y_train_encoded)

# Make predictions on the test data
y_pred_encoded = classifier.predict(X_test_emb)

# Decode the predictions
y_pred = label_encoder.inverse_transform(y_pred_encoded)

# Now you can use y_pred for further analysis or evaluation


In [48]:
# Création d'un DataFrame pour comparer les prédictions et les références
resultats = pd.DataFrame({
    'Titre': X_test,
    'Categorie Prédite': y_pred
})

# Réinitialisation de l'index pour une meilleure lisibilité
resultats.reset_index(drop=True, inplace=True)

# Affichage du tableau des résultats
print(resultats)

# Optionnel: Sauvegarder le tableau dans un fichier CSV
resultats.to_csv('../data/result_word2vec.csv', index=False)

y_true = df_test['type']
precision = accuracy_score(y_true, y_pred)
print(f'Précision du modèle : {precision} \n -----------------------------\n')

# Génération d'un rapport de classification
rapport_classification = classification_report(y_true, y_pred)
print('Rapport de classification :\n', rapport_classification)

rapport_df = pd.read_fwf(StringIO(rapport_classification), delimiter=';', header=0)
rapport_df.to_csv(f'../data/result/result_tfid_{now}.csv', index=False)

                                                  Titre Categorie Prédite
0                          Roulé à la confiture de lait    Plat principal
1                                Croissants aux amandes    Plat principal
2     Quinoa Phileas (aux légumes croquants et sauci...    Plat principal
3                   Magret de canard à la crème de mûre    Plat principal
4     St-Jacques a la sauce aux huitres et aux asperges    Plat principal
...                                                 ...               ...
1383             Galettes de pommes de terre aux navets    Plat principal
1384  Terrine d'aubergines au thon de Nadine (4ème r...    Plat principal
1385              Gateau aux amandes et fleur d'oranger    Plat principal
1386                               Cailles au vin rouge    Plat principal
1387  Gratin de pâtes au boeuf, courgettes et mascar...    Plat principal

[1388 rows x 2 columns]
Précision du modèle : 0.46397694524495675 
 -----------------------------

Rapport de c

/home/leon/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leon/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/leon/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
